In [1]:
import sys
import subprocess
import importlib
import os
import struct
import socket
import time
import tkinter as tk
from tkinter import simpledialog, filedialog, messagebox

# ---------------------------------------------------------
# 1. 패키지 자동 설치 함수
# ---------------------------------------------------------
def install_package(module_name, package_name=None):
    if package_name is None:
        package_name = module_name
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Installing {package_name} ...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
            print(f"{package_name} installation completed")
        except subprocess.CalledProcessError as e:
            print(f"{package_name} installation failed (exit code {e.returncode})")
            sys.exit(1)

print("Checking required packages...")
install_package("numpy")
install_package("cv2", "opencv-python")
install_package("tensorflow")

import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# ---------------------------------------------------------
# 2. 설정 및 연결 (IP 입력 팝업 추가)
# ---------------------------------------------------------
HOST_CAM = '192.168.137.220'  # 기본 IP
PORT_CAM = 80
PORT_MOT = 81

client_cam = None
client_mot = None

# Tkinter 루트 윈도우 생성 (숨김 상태)
root = tk.Tk()
root.withdraw() 

while True:
    try:
        print(f"Connecting to {HOST_CAM}...")
        
        # 소켓 생성
        client_cam = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_mot = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        # 딜레이 방지 옵션
        client_mot.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
        
        # 타임아웃 설정 (3초 안에 연결 안되면 실패 처리)
        client_cam.settimeout(3)
        client_mot.settimeout(3)
        
        client_cam.connect((HOST_CAM, PORT_CAM))
        client_mot.connect((HOST_CAM, PORT_MOT))
        
        # 연결 성공 시 타임아웃 해제 (데이터 수신 대기 위해)
        client_cam.settimeout(None)
        client_mot.settimeout(None)
        
        print("Connected successfully!")
        break # 반복문 탈출

    except Exception as e:
        print(f"Connection Failed: {e}")
        
        # 팝업창 띄우기
        new_ip = simpledialog.askstring("Connection Failed", 
                                      f"Failed to connect to {HOST_CAM}.\n\nEnter ESP32 IP Address:",
                                      parent=root,
                                      initialvalue=HOST_CAM)
        
        if new_ip:
            HOST_CAM = new_ip # 입력받은 IP로 업데이트 후 재시도
        else:
            print("User cancelled connection.")
            sys.exit()

# ---------------------------------------------------------
# 3. 모델 로드 (파일 선택 팝업 추가)
# ---------------------------------------------------------
model_path = 'model.h5'

# 모델 파일이 없으면 파일 선택창 띄우기
if not os.path.exists(model_path):
    print(f"'{model_path}' not found. Please select a model file.")
    
    # 파일 선택창을 최상단으로 띄우기
    root.lift()
    root.attributes('-topmost', True)
    
    model_path = filedialog.askopenfilename(title="Select AI Model File",
                                            filetypes=[("H5 Files", "*.h5"), ("All Files", "*.*")])
    
    # 선택 후 최상단 속성 해제 (다른 창 가림 방지)
    root.attributes('-topmost', False)

    if not model_path:
        print("No model selected. Exiting...")
        sys.exit()

print(f"Loading model: {model_path} ...")
try:
    model = load_model(model_path, compile=False)
    print("Model loaded.")
except Exception as e:
    messagebox.showerror("Error", f"Failed to load model:\n{e}")
    sys.exit()

# Tkinter 종료 (이제 필요 없음)
root.destroy()

names = ['_0_forward', '_1_right', '_2_left', '_3_stop']
cmd_chars = [b'F', b'R', b'L', b'S'] 

# ---------------------------------------------------------
# 4. 주행 루프
# ---------------------------------------------------------
t_prev = time.time()
cnt_frame = 0

try:
    while True:
        # 1. 영상 요청
        cmd = 12
        try:
            client_cam.sendall(struct.pack('B', cmd))
        except Exception:
            print("Connection lost (Send)")
            break

        # 2. 데이터 길이 수신
        try:
            data_len_bytes = client_cam.recv(4)
            if not data_len_bytes: break
            data_len = struct.unpack('I', data_len_bytes)[0]
        except Exception:
            print("Connection lost (Recv Header)")
            break

        # 3. 영상 데이터 수신
        img_data = b''
        while len(img_data) < data_len:
            packet = client_cam.recv(data_len - len(img_data))
            if not packet: break
            img_data += packet

        if len(img_data) < data_len:
            break

        # 4. 이미지 디코딩
        np_data = np.frombuffer(img_data, dtype='uint8')
        frame = cv2.imdecode(np_data, 1)
        
        if frame is None: continue

        # 화면 출력
        cv2.imshow('AI Driving View', cv2.resize(frame, (320, 240)))

        # 5. 전처리 (160x120, 0~1 정규화)
        image_resized = cv2.resize(frame, (160, 120)) 
        image_norm = image_resized.astype(np.float32) / 255.0
        image_tensor = np.expand_dims(image_norm, axis=0)

        # 6. 추론
        y_predict = model.predict(image_tensor, verbose=0)
        idx = np.argmax(y_predict, axis=1)[0]
        
        print(f"Pred: {names[idx]} ({y_predict[0][idx]:.2f})")

        # 7. 모터 제어 (문자 전송)
        try:
            client_mot.sendall(cmd_chars[idx])
        except Exception:
            print("Motor Connection lost")
            break

        key = cv2.waitKey(1)
        if key == 27: # ESC
            break

        cnt_frame += 1
        t_now = time.time()
        if t_now - t_prev > 1.0:
            print(f"FPS: {cnt_frame}")
            t_prev = t_now
            cnt_frame = 0

except KeyboardInterrupt:
    print("Stopping...")

finally:
    if client_cam: client_cam.close()
    if client_mot: client_mot.close()
    cv2.destroyAllWindows()

Checking required packages...
Connecting to 192.168.137.220...
Connection Failed: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
import sys
import subprocess
import importlib
import os
import struct
import socket
import time
import json # 설정 파일 로드용
import tkinter as tk
from tkinter import simpledialog, filedialog, messagebox

# ---------------------------------------------------------
# 1. 패키지 자동 설치
# ---------------------------------------------------------
def install_package(module_name, package_name=None):
    if package_name is None:
        package_name = module_name
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Installing {package_name} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

print("Checking required packages...")
install_package("numpy")
install_package("cv2", "opencv-python")
install_package("tensorflow")

import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# ---------------------------------------------------------
# 2. 설정 및 연결
# ---------------------------------------------------------
HOST_CAM = '192.168.0.60' # [주의] 초기 IP
PORT_CAM = 80
PORT_MOT = 81

client_cam = None
client_mot = None

# Tkinter (IP 입력 및 파일 선택용)
root = tk.Tk()
root.withdraw() 

# 소켓 연결 루프
while True:
    try:
        print(f"Connecting to {HOST_CAM}...")
        client_cam = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_mot = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        # 반응 속도 최적화
        client_mot.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
        
        client_cam.settimeout(3)
        client_mot.settimeout(3)
        
        client_cam.connect((HOST_CAM, PORT_CAM))
        client_mot.connect((HOST_CAM, PORT_MOT))
        
        client_cam.settimeout(None)
        client_mot.settimeout(None)
        print("Connected successfully!")
        break 
    except Exception as e:
        print(f"Connection Failed: {e}")
        new_ip = simpledialog.askstring("Connection Failed", 
                                      f"Failed to connect to {HOST_CAM}.\n\nEnter ESP32 IP Address:",
                                      parent=root,
                                      initialvalue=HOST_CAM)
        if new_ip: HOST_CAM = new_ip
        else: sys.exit()

# ---------------------------------------------------------
# 3. 모델 로드
# ---------------------------------------------------------
model_path = 'model.h5'

# 모델 파일 없으면 선택창 띄우기
if not os.path.exists(model_path):
    root.lift(); root.attributes('-topmost', True)
    model_path = filedialog.askopenfilename(title="Select AI Model File", filetypes=[("H5 Files", "*.h5")])
    root.attributes('-topmost', False)
    
    if not model_path:
        print("No model selected."); sys.exit()

try:
    print(f"Loading model: {model_path} ...")
    model = load_model(model_path, compile=False)
    print("Model loaded.")
except Exception as e:
    messagebox.showerror("Error", f"Failed to load model:\n{e}")
    sys.exit()

root.destroy()

names = ['_0_forward', '_1_right', '_2_left', '_3_stop']
cmd_chars = [b'F', b'R', b'L', b'S'] 

# ---------------------------------------------------------
# 4. 마스킹 설정 로드 및 트랙바 초기화
# ---------------------------------------------------------
CONFIG_FILE = "mask_config.json"
crop_top, crop_bottom, crop_left, crop_right = 0, 0, 0, 0

# 저장된 설정 불러오기
if os.path.exists(CONFIG_FILE):
    try:
        with open(CONFIG_FILE, 'r') as f:
            config = json.load(f)
            crop_top = config.get("top", 0)
            crop_bottom = config.get("bottom", 0)
            crop_left = config.get("left", 0)
            crop_right = config.get("right", 0)
        print(f"Loaded mask config: {config}")
    except:
        print("Failed to load mask config. Using default (0).")

def nothing(x): pass

WINDOW_NAME = 'AI Driving View (Masking Control)'
cv2.namedWindow(WINDOW_NAME)

# 트랙바 생성 (초기값은 불러온 설정값 적용)
cv2.createTrackbar('Top', WINDOW_NAME, crop_top, 240, nothing)
cv2.createTrackbar('Bottom', WINDOW_NAME, crop_bottom, 240, nothing)
cv2.createTrackbar('Left', WINDOW_NAME, crop_left, 320, nothing)
cv2.createTrackbar('Right', WINDOW_NAME, crop_right, 320, nothing)

# ---------------------------------------------------------
# 5. 주행 루프
# ---------------------------------------------------------
t_prev = time.time()
cnt_frame = 0

# 데이터 수집 때와 해상도 통일 (중요!)
DISPLAY_WIDTH = 640
DISPLAY_HEIGHT = 480

try:
    while True:
        # 1. 영상 수신
        try:
            client_cam.sendall(struct.pack('B', 12))
            data_len_bytes = client_cam.recv(4)
            if not data_len_bytes: break
            data_len = struct.unpack('I', data_len_bytes)[0]
            
            img_data = b''
            while len(img_data) < data_len:
                packet = client_cam.recv(data_len - len(img_data))
                if not packet: break
                img_data += packet
            if len(img_data) < data_len: break
            
            np_data = np.frombuffer(img_data, dtype='uint8')
            frame = cv2.imdecode(np_data, 1)
            if frame is None: continue

        except Exception:
            print("Connection Error")
            break

        # 2. 리사이즈 (640x480) - 마스킹 기준 해상도
        frame_resized = cv2.resize(frame, (DISPLAY_WIDTH, DISPLAY_HEIGHT))

        # 3. 트랙바 값 읽어서 마스킹 적용 (실시간 조절 가능)
        c_top = cv2.getTrackbarPos('Top', WINDOW_NAME)
        c_bottom = cv2.getTrackbarPos('Bottom', WINDOW_NAME)
        c_left = cv2.getTrackbarPos('Left', WINDOW_NAME)
        c_right = cv2.getTrackbarPos('Right', WINDOW_NAME)

        h, w, _ = frame_resized.shape
        if c_top > 0: frame_resized[:c_top, :] = 0
        if c_bottom > 0: frame_resized[h-c_bottom:, :] = 0
        if c_left > 0: frame_resized[:, :c_left] = 0
        if c_right > 0: frame_resized[:, w-c_right:] = 0

        # 화면 출력
        cv2.imshow(WINDOW_NAME, frame_resized)

        # 4. AI 모델 입력용 전처리
        # (마스킹된 이미지를 160x120으로 줄여서 추론)
        model_input = cv2.resize(frame_resized, (160, 120))
        image_norm = model_input.astype(np.float32) / 255.0
        image_tensor = np.expand_dims(image_norm, axis=0)

        # 5. 추론
        y_predict = model.predict(image_tensor, verbose=0)
        idx = np.argmax(y_predict, axis=1)[0]
        
        # 콘솔 출력 (확률 포함)
        print(f"Pred: {names[idx]} ({y_predict[0][idx]:.2f})")

        # 6. 모터 제어
        try:
            client_mot.sendall(cmd_chars[idx])
        except:
            pass

        key = cv2.waitKey(1)
        if key == 27: break

        cnt_frame += 1
        if time.time() - t_prev > 1.0:
            print(f"FPS: {cnt_frame}")
            t_prev = time.time()
            cnt_frame = 0

except KeyboardInterrupt:
    print("Stopping...")
finally:
    if client_cam: client_cam.close()
    if client_mot: client_mot.close()
    cv2.destroyAllWindows()